In [12]:
 # -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
import requests 
from tqdm import tqdm
import urllib.request 
from socket import error as SocketError
import errno
import pandas as pd

In [7]:
headers = 헤더추가

title_list = []
date_list = []
text_list = []
link_list = []
section_list=[]
url_list = []

target = ['문화','정치','사회','경제']

for page in tqdm(range(366,2971)): #2020.12.31 이하의 자료만 필요
    
    url = f'https://search.khan.co.kr/search.html?stb=khan&dm=0&q=%EC%B2%AD%EB%85%84&pg={page}&sort=1'
    req = requests.get(url,headers = headers)#headers = headers
    soup = BeautifulSoup(req.content, 'html.parser')

    #링크 가져오기 
    for contents_type in soup.select('dd.date > strong'):
        if contents_type.get_text() in target:
            section_list.append(contents_type.get_text()) #섹션 추가
            link1 = contents_type.parent.parent
            link2 = link1.select_one('dt > a')
            url_list.append(link2['href']) #타겟이 되는 섹션의 링크만 추가

            
            url = urllib.request.Request(link2['href'],headers=headers)
            try: #Connection reset by peer -exception
                res = urllib.request.urlopen(url).read() 
                soup = BeautifulSoup(res,  "html.parser", from_encoding='utf-8')

                #제목
                title = soup.find_all('h1', {'class': 'headline'}) if soup.find_all('p', {'class': 'headline'}) is not None else soup.find_all('h1', {'class': 'article_head'})
                
                #날짜
                date = soup.select('em')[0].get_text()[5:15]

                 #본문 
                text_cont = soup.find('div', {'class' : 'art_body'}).text.replace('\n', "")
                text_list.append(text_cont)
                
            
                try:    
                    title_list.append(title[0].text) #IndexError 예외처리
                    date_list.append(date)
                    link_list.append(url)

                except IndexError:      
                    date_list.append('예외')
                    title_list.append('예외')
                    text_list.append('예외')
                    
            except SocketError as e:
                date_list.append('예외')
                title_list.append('예외')
                text_list.append('예외')
                if e.errno != errno.ECONNRESET:
                    raise # Not error we are looking for
            pass # Handle error here.


  0%|          | 0/13 [00:00<?, ?it/s]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
  8%|▊         | 1/13 [00:02<00:29,  2.42s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
 15%|█▌        | 2/13 [00:05<00:28,  2.61s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be d

In [10]:
print(len(link_list))
print(len(section_list))
print(len(text_list))
print(len(date_list))
print(len(title_list))

80
81
81
81
81


In [13]:
df = pd.DataFrame({'날짜':date_list[:80],
                  '언론사명':'경향신문',
                  '섹션':section_list[:80],
                  '제목':title_list[:80],
                  '내용':text_list[:80],
                  'url':url_list[:80]})

df['내용'] = df['내용'].apply(lambda x : x.replace('\n',''))

df.to_csv('경로.csv', index = False, encoding = 'utf-8-sig')